In [0]:
# Creating a csv file for testing 

dbutils.fs.put("scenerios/skiplines.csv","""
line1
line2
line3
id,name,loc
1,ravi,banglore
2,raj,chennai
3,prasad,pune
4,sekhar,chennai""",True)

Wrote 91 bytes.
Out[33]: True

In [0]:
df = spark.read.csv("/scenerios/skiplines.csv",header=True)
display(df)
# Here we will see junk rows above header row

line1
line2
line3
id
1
2
3
4


In [0]:

rdd = sc.textFile("/scenerios/skiplines.csv")
rdd_final= rdd.zipWithIndex().filter(lambda x:x[1]>3).map(lambda x:x[0].split(","))
rdd_final.collect()



Out[35]: [['id', 'name', 'loc'],
 ['1', 'ravi', 'banglore'],
 ['2', 'raj', 'chennai'],
 ['3', 'prasad', 'pune'],
 ['4', 'sekhar', 'chennai']]

In [0]:
# Now make the first row as header and store in 'columns '
columns = rdd_final.collect()[0]
columns

skip_first_line = rdd_final.first()  # This will store the first row (not the header)
skip_first_line



Out[36]: ['id', 'name', 'loc']

In [0]:
data = rdd_final.toDF(columns).show() 

## !!! Still we see first row as header which is not correct 


+---+------+--------+
| id|  name|     loc|
+---+------+--------+
| id|  name|     loc|
|  1|  ravi|banglore|
|  2|   raj| chennai|
|  3|prasad|    pune|
|  4|sekhar| chennai|
+---+------+--------+



In [0]:
# Now from df , remove the first row 
# We can apply filer to remove the first header row
data = rdd_final.filter(lambda x:x!=skip_first_line).toDF(columns).show()   


+---+------+--------+
| id|  name|     loc|
+---+------+--------+
|  1|  ravi|banglore|
|  2|   raj| chennai|
|  3|prasad|    pune|
|  4|sekhar| chennai|
+---+------+--------+



In [0]:
# Finally create a final dataframe of all cleansed data 
final_df = rdd_final.filter(lambda x:x!=skip_first_line).toDF(columns)
display(final_df)

id,name,loc
1,ravi,banglore
2,raj,chennai
3,prasad,pune
4,sekhar,chennai
